# Бинарная классификация с применением нейронной сети с одним скрытым слоем

В данной лабораторной работе рассмотрим процесс создание нейронной сети с одним скрытым слоем. Сравните данную модель с уже изученной Вами ранее- логистической регрессией.

**В результате выполнения данной лабораторной Вы:**
- Создадите нейронную сеть с одним скрытым слоем для бинарной классификации
- Используете нелинейную функцию активации нейронов, например тангенциальную
- Рассчитаете ошибку при помощи перекресной энтропии
- Примените алгоритм обратного распространения ошибки


Создайте файл с именем **planar_utils.py** в корне папки с программой. Вставьте представленный ниже код:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import sklearn.datasets
import sklearn.linear_model

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y.ravel(), cmap=plt.cm.Spectral)


def sigmoid(x):
    """
    Compute the sigmoid of x

    Arguments:
    x -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def load_planar_dataset():
    np.random.seed(1)
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j

    X = X.T
    Y = Y.T

    return X, Y

def load_extra_datasets():
    N = 200
    noisy_circles = sklearn.datasets.make_circles(n_samples=N, factor=.5, noise=.3)
    noisy_moons = sklearn.datasets.make_moons(n_samples=N, noise=.2)
    blobs = sklearn.datasets.make_blobs(n_samples=N, random_state=5, n_features=2, centers=6)
    gaussian_quantiles = sklearn.datasets.make_gaussian_quantiles(mean=None, cov=0.5, n_samples=N, n_features=2, n_classes=2, shuffle=True, random_state=None)
    no_structure = np.random.rand(N, 2), np.random.rand(N, 2)

    return noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure

Создайте файл с именем **testCases.py** в корне папки с программой. Вставьте представленный ниже код:

In [ ]:
import numpy as np

def layer_sizes_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(5, 3)
    Y_assess = np.random.randn(2, 3)
    return X_assess, Y_assess

def initialize_parameters_test_case():
    n_x, n_h, n_y = 2, 4, 1
    return n_x, n_h, n_y


def forward_propagation_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    b1 = np.random.randn(4,1)
    b2 = np.array([[ -1.3]])

    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': b1,
     'b2': b2}

    return X_assess, parameters

def compute_cost_test_case():
    np.random.seed(1)
    Y_assess = (np.random.randn(1, 3) > 0)
    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
     'b2': np.array([[ 0.]])}

    a2 = (np.array([[ 0.5002307 ,  0.49985831,  0.50023963]]))

    return a2, Y_assess, parameters

def backward_propagation_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    Y_assess = (np.random.randn(1, 3) > 0)
    parameters = {'W1': np.array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
     'W2': np.array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
     'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
     'b2': np.array([[ 0.]])}

    cache = {'A1': np.array([[-0.00616578,  0.0020626 ,  0.00349619],
         [-0.05225116,  0.02725659, -0.02646251],
         [-0.02009721,  0.0036869 ,  0.02883756],
         [ 0.02152675, -0.01385234,  0.02599885]]),
  'A2': np.array([[ 0.5002307 ,  0.49985831,  0.50023963]]),
  'Z1': np.array([[-0.00616586,  0.0020626 ,  0.0034962 ],
         [-0.05229879,  0.02726335, -0.02646869],
         [-0.02009991,  0.00368692,  0.02884556],
         [ 0.02153007, -0.01385322,  0.02600471]]),
  'Z2': np.array([[ 0.00092281, -0.00056678,  0.00095853]])}
    return parameters, cache, X_assess, Y_assess

def update_parameters_test_case():
    parameters = {'W1': np.array([[-0.00615039,  0.0169021 ],
        [-0.02311792,  0.03137121],
        [-0.0169217 , -0.01752545],
        [ 0.00935436, -0.05018221]]),
 'W2': np.array([[-0.0104319 , -0.04019007,  0.01607211,  0.04440255]]),
 'b1': np.array([[ -8.97523455e-07],
        [  8.15562092e-06],
        [  6.04810633e-07],
        [ -2.54560700e-06]]),
 'b2': np.array([[  9.14954378e-05]])}

    grads = {'dW1': np.array([[ 0.00023322, -0.00205423],
        [ 0.00082222, -0.00700776],
        [-0.00031831,  0.0028636 ],
        [-0.00092857,  0.00809933]]),
 'dW2': np.array([[ -1.75740039e-05,   3.70231337e-03,  -1.25683095e-03,
          -2.55715317e-03]]),
 'db1': np.array([[  1.05570087e-07],
        [ -3.81814487e-06],
        [ -1.90155145e-07],
        [  5.46467802e-07]]),
 'db2': np.array([[ -1.08923140e-05]])}
    return parameters, grads

def nn_model_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    Y_assess = (np.random.randn(1, 3) > 0)
    return X_assess, Y_assess

def predict_test_case():
    np.random.seed(1)
    X_assess = np.random.randn(2, 3)
    parameters = {'W1': np.array([[-0.00615039,  0.0169021 ],
        [-0.02311792,  0.03137121],
        [-0.0169217 , -0.01752545],
        [ 0.00935436, -0.05018221]]),
     'W2': np.array([[-0.0104319 , -0.04019007,  0.01607211,  0.04440255]]),
     'b1': np.array([[ -8.97523455e-07],
        [  8.15562092e-06],
        [  6.04810633e-07],
        [ -2.54560700e-06]]),
     'b2': np.array([[  9.14954378e-05]])}
    return parameters, X_assess

## 1 - Библиотеки ##

Вначале импортируем следующие необходимые для выполнения лабораторной работы библиотеки:
- [numpy](https://www.numpy.org/) - одна из основных библиотек для работы с данными при использовании языка Python.
- [sklearn](http://scikit-learn.org/stable/) предоставляет простые и эффективные модели для анализа данных
- [matplotlib](http://matplotlib.org) - это библиотека для создания графиков.
- Модуль testCases содержит набор тестовых примеров и позволяет оценить корректность реализованных Вами функций.
- Модуль planar_utils содержит множество функций используемых в данной лабораторной работе.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from testCases import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

%matplotlib inline

np.random.seed(1) # не меняйте это значение для автоматической проверки правильности выполнения

## 2 - Набор данных ##

Для начала загрузим данные, которые необходимо анализировать. Следующий код загружает набор данных "Полярная роза" (известная математическая кривая, похожая на цветок с лепестками). Загрузим координаты точек `X` и `Y`.

In [ ]:
X, Y = load_planar_dataset()

Визуализируем данные с помощью библиотеки matplotlib. Точки представляют собой "цветок" и имеют красные маркеры (y=0) и синие (y=1). Необходимо создать модель классификации точек по цвету.

In [ ]:
plt.scatter(X[0, :], X[1, :], c=Y.ravel(), s=40, cmap=plt.cm.Spectral)

 Имеем:
    - numpy-массив (матрица) X содержащая переменные (x1, x2)
    - numpy-массив (вектор) Y содержащий метку класса (красный:0, синий:1).


**Задание**: Каков размер вашей обучающей выборки? А также,какова форма numpy-массивов `X` и `Y`?


YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

print ('Форма X: ' + str(shape_X))
print ('Форма Y: ' + str(shape_Y))
print ('m = ',m)

In [ ]:
assert shape_X == (2,400)
assert shape_Y == (1,400)
assert m == 400

## 3 - Логистическая регрессия

Перед созданием нейронной сети рассмотрим применимость логистической регрессии в данном случае. Для этого Вы можете использовать встроенные функции библиотеки sklearn. Запустите следующий код для обучения классификатора на основе логистической регрессии по представленным данным.

In [ ]:
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);

Теперь вы можете визуализировать границу раздела классов для данной модели. Запустите код? предложенный ниже.

In [ ]:
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Логистическая регрессия")

LR_predictions = clf.predict(X.T)
print ('Точность логистической регрессии: ', float((np.dot(Y,LR_predictions) + np.dot(1-Y,1-LR_predictions))/float(Y.size)*100),
       '% (процент правильно классифицированных точек)')

**Задание**: Объясните низкий процент точности при применении логистической регреессии к данной задаче.

YOUR ANSWER HERE

## 4 - Нейронная сеть

Алгоритм логистичской регрессии показал слабую классификацию на наборе данных "Полярная роза". Далее Вам предстоить использовать нейронную сеть с одним скрытым слоем для улучшения качества класссификации.

**Архитектура используемой сети с одним скрытым слоем**:
<img src="https://drive.google.com/uc?id=1brDmfJ-irjQTnLe_4fzjBXRsNZo9ghzc" width=70%>

**Алгоритм обучения нейронной сети**:

Для экземпляра данных $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{, если } a^{[2](i)} > 0.5 \\ 0 & \mbox{, иначе } \end{cases}\tag{5}$$

Используя все обучающие примеры, в качестве функции ошибки ИНС будем оценивать функционал стоимости $J$ как:
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Важно**: Используем алогоритм построения искусственной нейронной сети:
    1. Определить архитектуру нейросетевой модели ( количество входов,  число скрытых слоев, размеры слоев и другие).
    2. Инициализировать параметры модели (веса, смещения, граденты, оценку функционала и другие)
    3. На каждой итерации обучения:
        - Прямое распространение сигнала по сети (предъявление очередного
        набора из обучающей выборки на вход нейронной сети)
        - Вычисление выходного сигнала сети и функции ошибки
        - Обратное распространение ошибки и определение величин ошибок нейронов
        - Обновление весов сети (методом градиентного спуска)

В начале следует реализовать основные функции алгоритма, а затем использовать их в правильном порядке в главной функции модели `nn_model()`. После реализации `nn_model()` и обучения нейронной сети Вы получите классификатор и сможете его применить на "новых" для сети данных (которых не было в обучающем наборе).

### 4.1 - Определение архитектуры нейронной сети ####

**Задание**: Определите три переменных в соотвествии с изображением:
    - n_x: размер входного слоя
    - n_h: размер скрытого слоя
    - n_y: размер выходного слоя

In [ ]:
def layer_sizes(X, Y):
    # YOUR CODE HERE
    raise NotImplementedError()
    return (n_x, n_h, n_y)

In [ ]:
X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("Размер входного слоя: n_x = " + str(n_x))
print("Размер скрытого слоя: n_h = " + str(n_h))
print("Размер выходного слоя: n_y = " + str(n_y))

assert n_x == 5
assert n_h == 4
assert n_y == 2

### 4.2 - Инициализация параметров модели ####

**Задание**: Реализуйте функцию `initialize_parameters()`, которая инициализирует веса W нейронов вещественными случайными числами и смещения b нулями.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(2) # не меняйте значение для автопроверки

    # YOUR CODE HERE
    raise NotImplementedError()

    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### 4.3 - Цикл обучения ####

**Задание**: Реализуйте функцию прямого распространения сигнала по сети `forward_propagation()`.

**Указания**:
- Следуйте математическому описанию работы нейрона.
- Используйте функцию `sigmoid()` из пакета planar_utils и тангенциальную функцию np.tanh().
- Реализуйте прямое распространение обучающей выборки по сети, вычислив векторы выходных значений и активаций нейронов $Z^{[1]}, A^{[1]}, Z^{[2]}$ и $A^{[2]}$.
- Значения для вышисления ошибок при обратном распространении ошибок сохраняются в "`cache`".

In [ ]:
def forward_propagation(X, parameters):
    # YOUR CODE HERE
    raise NotImplementedError()

    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}

    return A2, cache

In [ ]:
X_assess, parameters = forward_propagation_test_case()
A2, cache = forward_propagation(X_assess, parameters)

print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))

assert(A2.shape == (1, X_assess.shape[1]))

После вычисления вектора $A^{[2]}$ выходных значений нейрона выходного слоя для каждого экземпляра обучающей выборки $a^{[2](i)}$, необходимо вычислить функционал стоимости (ошибки) на основе перекрестной энтропии:

$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{13}$$

**Задание**: Реализуйте функцию `compute_cost()` для расчета функционала ошибки $J$ с помощью.

In [ ]:
def compute_cost(A2, Y, parameters):
    m = Y.shape[1]

    # YOUR CODE HERE
    raise NotImplementedError()

    return cost

In [ ]:
A2, Y_assess, parameters = compute_cost_test_case()

cost = compute_cost(A2, Y_assess, parameters)
print("cost = " + str(cost))

assert(isinstance(cost, float))

**Задание**: Используя сохраненные значения при прямом проходе, реализуйте обратное распространение ошибки `backward_propagation()`.

**Указания**:
Минимизация методом градиентного спуска обеспечивает подстройку весовых коэффициентов следующим образом:  

<img src="https://drive.google.com/uc?id=1KH_eI13-RlGOvp0OPyRGom4tlxCxHWof" width=80%>

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
    
!-->

- Совет:
    - Для вычисления dZ1 необходимо вычислить $g^{[1]'}(Z^{[1]})$. Т.к. $g^{[1]}(.)$ - тангенциальная функция активации, если $a = g^{[1]}(z)$, то $g^{[1]'}(z) = 1-a^2$. Так вы можете вычислить
    $g^{[1]'}(Z^{[1]})$ используя `(1 - np.power(A1, 2))`.

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[1]

    # YOUR CODE HERE
    raise NotImplementedError()

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}

    return grads

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Проверочные значения**:



<table style="width:80%">
  <tr>
    <td>**dW1**</td>
    <td> [[ 0.00301023 -0.00747267]
 [ 0.00257968 -0.00641288]
 [-0.00156892  0.003893  ]
 [-0.00652037  0.01618243]] </td>
  </tr>
  
  <tr>
    <td>**db1**</td>
    <td>  [[ 0.00176201]
 [ 0.00150995]
 [-0.00091736]
 [-0.00381422]] </td>
  </tr>
  
  <tr>
    <td>**dW2**</td>
    <td> [[ 0.00078841  0.01765429 -0.00084166 -0.01022527]] </td>
  </tr>
  

  <tr>
    <td>**db2**</td>
    <td> [[-0.16655712]] </td>
  </tr>
  
</table>  

**Задание**: Реализуйте коррекцию весов, используя (dW1, db1, dW2, db2) для обновления (W1, b1, W2, b2).

**Правило обновления весов градиентного спуска**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$, где $\alpha$ - это параметр скорости обучения, а $\theta$ представляет обновляемый параметр.

Алгоритм градиентного спуска с оптимальной скорость обучения сходится, а с плохо подобранной - расходится.

<img src="https://drive.google.com/uc?id=17C5-UNXd2Ec2aYKU_DtosW_WkDXwyGGk" style="width:400;height:400;"> <img src="https://drive.google.com/uc?id=1ePjQsFSElPrwIZtaqFVvquvm-d7Q2_2a" style="width:400;height:400;">



In [ ]:
def update_parameters(parameters, grads, learning_rate = 1.2):
    # YOUR CODE HERE
    raise NotImplementedError()

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Ожидаемые значения**:


<table style="width:80%">
  <tr>
    <td>**W1**</td>
    <td> [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]</td>
  </tr>
  
  <tr>
    <td>**b1**</td>
    <td> [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]</td>
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[-0.01041081 -0.04463285  0.01758031  0.04747113]] </td>
  </tr>
  

  <tr>
    <td>**b2**</td>
    <td> [[ 0.00010457]] </td>
  </tr>
  
</table>  

### 4.4 - Построение модели нейронной сети nn_model() ####

**Задание**: Постройте нейросетевую модель в функции `nn_model()`, используя реализованные функции.

In [ ]:
def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):

    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]

    # parameters =
    # YOUR CODE HERE
    raise NotImplementedError()

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # YOUR CODE HERE
        raise NotImplementedError()

        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Значение ошибки после %i итерации: %f" %(i, cost))

    return parameters

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

**Ожидаемые значения**:

<table style="width:90%">

<tr>
    <td>
        **cost after iteration 0**
    </td>
    <td>
        0.692739
    </td>
</tr>

<tr>
    <td>
        <center> $\vdots$ </center>
    </td>
    <td>
        <center> $\vdots$ </center>
    </td>
</tr>

  <tr>
    <td>**W1**</td>
    <td> [[-0.65848169  1.21866811]
 [-0.76204273  1.39377573]
 [ 0.5792005  -1.10397703]
 [ 0.76773391 -1.41477129]]</td>
  </tr>
  
  <tr>
    <td>**b1**</td>
    <td> [[ 0.287592  ]
 [ 0.3511264 ]
 [-0.2431246 ]
 [-0.35772805]] </td>
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[-2.45566237 -3.27042274  2.00784958  3.36773273]] </td>
  </tr>
  

  <tr>
    <td>**b2**</td>
    <td> [[ 0.20459656]] </td>
  </tr>
  
</table>  

### 4.5 Прогноз

**Задание**: Реализуйте функцию predict() для прогноза модели.
Используйте прямое распространение сигнала по сети.


In [ ]:
def predict(parameters, X):
    """
    Используя параметры обученной модели, предскажем класс для каждого примера в X

    Аргументы:
    parameters -- Словарь python хранит в себе параметры вашей модели
    X -- входные данные размерности (n_x, m)

    Выход функции
    predictions -- Вектор с прогнозом вашей модели (красный: 0 / синий: 1)
    """

    # YOUR CODE HERE
    raise NotImplementedError()

    return predictions

In [ ]:
parameters, X_assess = predict_test_case()

predictions = predict(parameters, X_assess)
print("predictions mean = " + str(np.mean(predictions)))

assert round(np.mean(predictions),3)==0.667

Давайте запустим модель и посмотрим как она справится со всем набором данных. Запустите следующий код и протестируйте свою нейронную сеть с одим скрытым слоем, содержащим $n_h$ нейронов.

In [ ]:
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
plt.title("Decision Boundary for hidden layer size " + str(4))

**Ожидаемый выход**:

<table style="width:40%">
  <tr>
    <td>**Cost after iteration 9000**</td>
    <td> 0.218607 </td>
  </tr>
  
</table>


In [ ]:
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

**Expected Output**:

<table style="width:15%">
  <tr>
    <td>**Accuracy**</td>
    <td> 90% </td>
  </tr>
</table>

Точность значительно увеличилась по сравнению с логистической регрессией, давайте теперь попробуем различные варианты реализации скрытого слоя.

### 4.6 - Изменение количество нейронов на скрытом слое ###

Запустите следующий код. Проанализируйте влияние размерности скрытого слоя на результат классификации.

In [ ]:
plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i+1)
    plt.title('Размер скрытого слоя %d' % n_h)
    parameters = nn_model(X, Y, n_h, num_iterations = 5000)
    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
    predictions = predict(parameters, X)
    accuracy = float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100)
    print ("Точность для {} скрытого слоя: {} %".format(n_h, accuracy))

**Объяснения**:
- Чем больше нейронов на скрытом слое, тем более точно модель описывает обучающую выборку, пока в какой-то момент максимально большая модель не переучится.
- Наиболее удачной будет модель с 5-ю нейронами на скрытом слое. При этом значении переобучение еще не происходит.
- Позже вы познакомитесь с понятием "регуляризация". Регуляризация позволяет использовать модели с большим количеством нейронов на скрытом слое без переобучения сети.

**Вопросы**:

- Что произойдет если изменить функцию активации с tanh на sigmoid или на ReLU?
- Проанализируйте на что влияет изменение learning_rate.
- Выберите датасет из раздела 5, согласно своему варианту.

YOUR ANSWER HERE

## 5) Самостоятельная работа.
Выберите датасет согласно своему варианту. Обучите нейронную сеть и выполните классификацию. Постройте график ошибки на обучении, подберите наилучшую скорость обучения и количество эпох обучения. Сделайте выводы. Испробуте другие варианты архитектуры и функций активации на платформе http://playground.tensorflow.org/

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

Контрольные вопросы:
1. Что входит в понятие архитектуры нейронной сети?
2. Какие параметры есть у нейронных сетей и как они могут быть инициализированы?
3. Что такое функционал стоимости? Для чего он нужен?
